In [45]:
import json
import os

import numpy as np
import pandas as pd
from IPython.display import Image

In [10]:
ROOT_DIR = os.getcwd()

DB_PATH = ROOT_DIR + "/db/"
YEAR = "2024"
UPLOAD_PATH = ROOT_DIR

TRASH_ID, TRASH_CATEGORY = -1, "trash"
_CATEGORY_INFO = json.load(open(ROOT_DIR + f"/{YEAR}.json"))
ID2CAT = {int(cat_id): cat for cat_id, cat in _CATEGORY_INFO.items()}
ID2CAT_ALL = {**ID2CAT, TRASH_ID: TRASH_CATEGORY}

VOTES_FILE = DB_PATH + "votes.csv"
IP_TO_USER_FILE = DB_PATH + "ip_to_user.csv"
USER_TO_IMAGE_FILE = DB_PATH + "user_to_image.csv"

In [51]:
def read_data(csv_file: str) -> pd.DataFrame:
    """Read data from a file. First line is the header. Index is not set."""
    return pd.read_csv(csv_file, header=0)


def idxmedian(series: pd.Series) -> str:
    """Return the index of the median value of the series."""
    return series.index[np.argsort(series.values)[len(series) // 2]]


def score_users():
    """Evaluate user scores."""
    uploads = read_data(USER_TO_IMAGE_FILE).set_index(["cat_id", "img_name"]).rename(columns={"user": "author"})
    votes = read_data(VOTES_FILE).set_index(["cat_id", "img_name"]).rename(columns={"user": "voter"})
    votes["both"] = votes.funny + votes.cringe

    votes = pd.merge(votes, uploads, left_index=True, right_index=True).reset_index()

    # aggregations = [
    #     # ("princesa_mediana", "both", idxmedian),
    #     ("grof_smehoslav", "funny", "idxmax"),
    #     ("skremžni_knez", "cringe", "idxmax"),
    #     ("meme_lord", "both", "idxmax"),
    # ]

    return (
        votes.groupby("author").sum()["cringe"].sort_values(ascending=False),
        votes.groupby("author").sum()["funny"].sort_values(ascending=False),
        votes.groupby("author").sum()["both"].sort_values(ascending=False),
    )


skremz, smeh, meme_lords = score_users()

In [50]:
def score_memes():
    """Evaluate meme scores."""
    votes = read_data(VOTES_FILE)
    votes["both"] = votes.funny + votes.cringe

    # aggregations = [
    #     ("jazjaz_notranje_bolečine", "cringe", "idxmax"),
    #     ("smesen_ful_majkemi", "both", "idxmax"),
    #     ("najnajjazjaz", "funny", "idxmax"),
    # ]

    cringe = votes.groupby(["cat_id", "img_name"]).sum()["cringe"].sort_values(ascending=False)[0:10].to_dict()
    funny = votes.groupby(["cat_id", "img_name"]).sum()["funny"].sort_values(ascending=False)[0:10]
    both = votes.groupby(["cat_id", "img_name"]).sum()["both"].sort_values(ascending=False)[0:10]

    cringe = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in cringe.items()]
    funny = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in funny.items()]
    both = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in both.items()]

    return cringe, funny, both


cringe, funny, both = score_memes()

print("CRINGE")
for img in cringe:
    print(img)
    Image(filename=img)

print("FUNNY")
for img in funny:
    print(img)
    Image(filename=img)

print("BOTH")
for img in both:
    print(img)
    Image(filename=img)

CRINGE
FUNNY
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/d80a81e3.png
/home/ubobek/ws/private/hehormeh/archive/2024/roast/ad40d2fb.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/58eef90d.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/4d5049d3.png
/home/ubobek/ws/private/hehormeh/archive/2024/roast/5dd73efa.jpg
/home/ubobek/ws/private/hehormeh/archive/2024/relatable/662f40a9.jpg
/home/ubobek/ws/private/hehormeh/archive/2024/roast/8849b105.png
/home/ubobek/ws/private/hehormeh/archive/2024/roast/bfa0c3b6.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/relatable/12323bd9.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/3e77d947.jpg
BOTH
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/d80a81e3.png
/home/ubobek/ws/private/hehormeh/archive/2024/roast/ad40d2fb.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/roast/bfa0c3b6.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/58eef90d.JPG
/home/ubo

In [52]:
def score_memes():
    """Evaluate meme scores."""
    votes = read_data(VOTES_FILE)
    votes["both"] = votes.funny + votes.cringe

    # aggregations = [
    #     ("jazjaz_notranje_bolečine", "cringe", "idxmax"),
    #     ("smesen_ful_majkemi", "both", "idxmax"),
    #     ("najnajjazjaz", "funny", "idxmax"),
    # ]

    # results = votes.groupby(["cat_id", "img_name"]).sum()["both"]

    cringe = votes.groupby(["cat_id", "img_name"]).sum()["cringe"].sort_values(ascending=False)[0:10].to_dict()
    funny = votes.groupby(["cat_id", "img_name"]).sum()["funny"].sort_values(ascending=False)[0:10]
    both = votes.groupby(["cat_id", "img_name"]).sum()["both"].sort_values(ascending=False)[0:10]

    cringe = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in cringe.items()]
    funny = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in funny.items()]
    both = [f"{UPLOAD_PATH}/{ID2CAT[img_data[0]]}/{img_data[1]}" for img_data, score in both.items()]

    return cringe, funny, both


cringe, funny, both = score_memes()

print("CRINGE")
for img in cringe:
    print(img)
    Image(filename=img)

print("FUNNY")
for img in funny:
    print(img)
    Image(filename=img)

print("BOTH")
for img in both:
    print(img)
    Image(filename=img)

CRINGE
/home/ubobek/ws/private/hehormeh/archive/2024/roast/fd64bab2.jpg
/home/ubobek/ws/private/hehormeh/archive/2024/freestyle/d363d665.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/13c3316e.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/relatable/fe12584c.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/367d6069.gif
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/8c77ff13.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/267f2b26.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/roast/bfa0c3b6.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/roast/a27ea2f9.jpeg
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/56daddf0.jpg
Funn
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/d80a81e3.png
/home/ubobek/ws/private/hehormeh/archive/2024/roast/ad40d2fb.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/ljubavna_izpoved/58eef90d.JPG
/home/ubobek/ws/private/hehormeh/archive/2024/sam_slabo/4d5049d3.png
/home/ubobe